## Dataset Transformation

* BTC: 01/01/2022, 02/01/2022, 31/12/2022 missing values
* ETH: 12/01/2018, 02/12/2020 missing values
* LTC: 12/01/2018, 02/12/2020 missing values
* XRP: 12/01/2018, 02/12/2020 missing values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Bitcoin  - BTC

In [ ]:
bitcoin= pd.read_csv('BTC.csv')
bitcoin.head()

,Crypto,Date,Open,High,Low,Close
0,BTC,12/31/19,7254.0,7309.0,7132.0,7171.0
1,BTC,12/30/19,7402.0,7430.0,7217.0,7254.0
2,BTC,12/29/19,7334.0,7529.0,7295.0,7402.0
3,BTC,12/28/19,7235.0,7359.0,7235.0,7334.0
4,BTC,12/27/19,7208.0,7267.0,7087.0,7235.0


In [ ]:
# --------------------------------
# Step 1 — Sort by ascending dates
# --------------------------------

bitcoin['Date'] = pd.to_datetime(bitcoin['Date'], dayfirst=False)
bitcoin = bitcoin.sort_values(by='Date', ascending=True)
bitcoin['Date'] = bitcoin['Date'].dt.strftime('%d/%m/%Y')


/var/folders/2y/l32d6fw97yq94q7xx_4l4cl00000gn/T/ipykernel_4405/1172347968.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  bitcoin['Date'] = pd.to_datetime(bitcoin['Date'], dayfirst=False)


In [ ]:
bitcoin.tail()
## 1,974 datapoints but should be 1,977 need to check missing values and extrapolate

,Crypto,Date,Open,High,Low,Close
1465,BTC,27/05/2023,26752.0,26832.0,26629.0,26759.0
1464,BTC,28/05/2023,26759.0,27661.0,26747.0,27552.0
1463,BTC,29/05/2023,27552.0,28419.0,27537.0,27680.0
1462,BTC,30/05/2023,27680.0,28019.0,27573.0,27777.0
1461,BTC,31/05/2023,27777.0,27837.0,26884.0,27120.0


In [ ]:
bitcoin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1974 entries, 729 to 1461
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Crypto  1974 non-null   object 
 1   Date    1974 non-null   object 
 2   Open    1974 non-null   float64
 3   High    1974 non-null   float64
 4   Low     1974 non-null   float64
 5   Close   1974 non-null   float64
dtypes: float64(4), object(2)
memory usage: 108.0+ KB


In [ ]:
# -------------------------------------
# Step 2 — Determine missing date/value
# -------------------------------------

bitcoin['Date'] = pd.to_datetime(bitcoin['Date'], dayfirst=True)
full_range = pd.date_range(start='01/01/2018', end='31/05/2023')

diff = full_range.difference(bitcoin['Date'])
print("Missing dates in the dataset:", diff)
# btc: 01/01/2022, 02/01/2022, 31/12/2022 missing values


In [ ]:
# --------------------------
# Step 3 — Add missing dates
# --------------------------

bitcoin.set_index('Date', inplace=True)
bitcoin = bitcoin.reindex(full_range)

In [ ]:
# ----------------------------------------------------
# Step 4 — Linearly interpolate missing values (price)
# ----------------------------------------------------

bitcoin['Crypto']= 'BTC'
bitcoin.isnull().sum()
bitcoin.interpolate(method='linear', inplace=True)
bitcoin.reset_index(inplace=True)
bitcoin.rename(columns={'index': 'Date'}, inplace=True)

/var/folders/2y/l32d6fw97yq94q7xx_4l4cl00000gn/T/ipykernel_4405/383623647.py:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  bitcoin.interpolate(method='linear', inplace=True)


In [ ]:
# -----------------------------------------
# Step 5 — Export new DF (without scaling)
# -----------------------------------------

bitcoin.to_csv('BTC_noscale.csv', index=False)

In [ ]:
# --------------------------------
# Step 6 — Log returns calculation
# --------------------------------

bitcoin['Log_Returns'] = np.log(bitcoin['Close'] / bitcoin['Close'].shift(1))
bitcoin['Date'] = pd.to_datetime(bitcoin['Date'])
bitcoin.set_index('Date', inplace=True)

In [ ]:
# --------------------
# Step 7 — Volatility
# --------------------

monthly_vol = bitcoin.groupby(bitcoin.index.to_period('M'))['Log_Returns'].std() * np.sqrt(365)
bitcoin['Volatility'] = bitcoin.index.to_period('M').map(monthly_vol)

In [ ]:
# Fill with 0 will impact change in volatility so keep NaN
#bitcoin['Log_Returns'].fillna(0, inplace=True)

In [ ]:
bitcoin.head()

,Crypto,Open,High,Low,Close,Log_Returns,Volatility
Date,,,,,,,
2018-01-01,BTC,13996.0,14035.0,12860.0,13535.0,NaN,1.528575
2018-01-02,BTC,13535.0,15217.0,12956.0,14770.0,0.087319,1.528575
2018-01-03,BTC,14770.0,15394.0,14589.0,15057.0,0.019245,1.528575
2018-01-04,BTC,15057.0,15395.0,14225.0,14921.0,-0.009073,1.528575
2018-01-05,BTC,14921.0,16909.0,14817.0,16828.0,0.120275,1.528575


In [ ]:
# -----------------------------------------------
# Step 8  — Plot Dataset Features over the years
# -----------------------------------------------
# Plot Target variable i.e. BTC close price
plt.figure(figsize=(10,5))
plt.plot(btc['Close'], color='blue');

In [ ]:
figure, ax = plt.subplots(nrows=7, ncols=1, figsize=(12,10))
ax[0].set_ylabel('Close Price')
ax[1].set_ylabel('Open Price')
ax[2].set_ylabel('High Price')
ax[3].set_ylabel('Low Price')
ax[4].set_ylabel('Mean Price')
ax[5].set_ylabel('Volatility')
ax[6].set_ylabel('Log Returns')

ax[0].plot(btc['Close'], color='blue');
ax[1].plot(btc['Open'], color='red', alpha=0.75);
ax[2].plot(btc['High'], color='green');
ax[3].plot(btc['Low'], color='orange');
ax[4].plot(btc['Mean'], color='cyan');

ax[5].plot(btc['Volatility'], color='black');
ax[6].plot(btc['Log_Returns'], color='purple');

In [ ]:
# -----------------------------------
# Step 9 — MinMax scaling between 0-1
# -----------------------------------

btc_scaler = MinMaxScaler(feature_range=(0, 1))
bitcoin['Open'] = btc_scaler.fit_transform(bitcoin['Open'].values.reshape(-1, 1))
bitcoin['High'] = btc_scaler.fit_transform(bitcoin['High'].values.reshape(-1, 1))
bitcoin['Low'] = btc_scaler.fit_transform(bitcoin['Low'].values.reshape(-1, 1))
bitcoin['Close'] = btc_scaler.fit_transform(bitcoin['Close'].values.reshape(-1, 1))
bitcoin['Log_Returns'] = btc_scaler.fit_transform(bitcoin['Log_Returns'].values.reshape(-1, 1))
bitcoin['Volatility'] = btc_scaler.fit_transform(bitcoin['Volatility'].values.reshape(-1, 1))

bitcoin.head()

,Crypto,Open,High,Low,Close,Log_Returns,Volatility
Date,,,,,,,
2018-01-01,BTC,0.167270,0.164014,0.154139,0.160136,NaN,1.0
2018-01-02,BTC,0.160136,0.182016,0.155664,0.179249,0.852180,1.0
2018-01-03,BTC,0.179249,0.184712,0.181603,0.183691,0.706281,1.0
2018-01-04,BTC,0.183691,0.184727,0.175821,0.181586,0.645588,1.0
2018-01-05,BTC,0.181586,0.207786,0.185225,0.211100,0.922812,1.0


In [ ]:
# ------------------------------
# Step 10 — Export scaled dataset
# ------------------------------

bitcoin.to_csv('BTC_cleaned.csv', index=False)

### Ethereum - ETH

In [ ]:
eth = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ETH.csv')
eth.head()

,Crypto,Date,Open,High,Low,Close
0,ETH,12/31/21,3736.59,3812.67,3622.51,3687.67
1,ETH,12/30/21,3725.60,3768.63,3590.00,3733.98
2,ETH,12/29/21,3809.14,3827.01,3700.00,3724.98
3,ETH,12/28/21,4061.29,4075.97,3765.23,3807.82
4,ETH,12/27/21,4086.36,4128.28,4037.00,4061.30


In [ ]:
# --------------------------------
# Step 1 — Sort by ascending dates
# --------------------------------

eth['Date'] = pd.to_datetime(eth['Date'], dayfirst=False)
eth = eth.sort_values(by='Date', ascending=True)
eth['Date'] = eth['Date'].dt.strftime('%d/%m/%Y')


/tmp/ipython-input-4204808973.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  eth['Date'] = pd.to_datetime(eth['Date'], dayfirst=False)


In [ ]:
eth.tail()
## 1,975 datapoints instead of 1,977
eth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1977 entries, 0 to 1976
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1977 non-null   datetime64[ns]
 1   Crypto  1977 non-null   object        
 2   Open    1977 non-null   float64       
 3   High    1977 non-null   float64       
 4   Low     1977 non-null   float64       
 5   Close   1977 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 92.8+ KB


In [ ]:
# -------------------------------------
# Step 2 — Determine missing date/value
# -------------------------------------

eth['Date'] = pd.to_datetime(eth['Date'], dayfirst=True)

# Full range of dates
full_range = pd.date_range(start='2018-01-01', end='2023-05-31')
diff = full_range.difference(eth['Date'])

print("Missing dates in the dataset:", diff)
# eth: 12/01/2018, 02/12/2020 missing values

Missing dates in the dataset: DatetimeIndex(['2018-01-12', '2020-12-02'], dtype='datetime64[ns]', freq=None)


In [ ]:
# --------------------------
# Step 3 — Add missing dates
# --------------------------

eth.set_index('Date', inplace=True)
eth = eth.reindex(full_range)

In [ ]:
# ----------------------------------------------------
# Step 4 — Linearly interpolate missing values (price)
# ----------------------------------------------------

eth['Crypto']= 'ETH'
eth.isnull().sum()
eth.interpolate(method='linear', inplace=True)
eth.reset_index(inplace=True)
eth.rename(columns={'index': 'Date'}, inplace=True)

/tmp/ipython-input-4253183272.py:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  eth.interpolate(method='linear', inplace=True)


In [ ]:
eth.head()

,Date,Crypto,Open,High,Low,Close
0,2018-01-01,ETH,744.39,772.98,725.10,757.01
1,2018-01-02,ETH,758.00,879.90,757.01,864.00
2,2018-01-03,ETH,861.99,947.92,851.00,938.00
3,2018-01-04,ETH,937.18,988.99,905.01,942.04
4,2018-01-05,ETH,942.04,1009.84,900.00,958.00


In [ ]:
# -----------------------------------------
# Step 5 — Export new DF (without scaling)
# -----------------------------------------

eth.to_csv('ETH_noscale.csv', index=False)

from google.colab import files
files.download('ETH_noscale.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# -----------------------------------------------
# Step 6  — Plot Dataset Features over the years
# -----------------------------------------------
# Plot Target variable i.e. ETH close price
plt.figure(figsize=(10,5))
plt.plot(eth['Close'], color='blue');
figure, ax = plt.subplots(nrows=7, ncols=1, figsize=(12,10))
ax[0].set_ylabel('Close Price')
ax[1].set_ylabel('Open Price')
ax[2].set_ylabel('High Price')
ax[3].set_ylabel('Low Price')
ax[4].set_ylabel('Mean Price')
ax[5].set_ylabel('Volatility')
ax[6].set_ylabel('Log Returns')

ax[0].plot(eth['Close'], color='blue');
ax[1].plot(eth['Open'], color='red', alpha=0.75);
ax[2].plot(eth'High'], color='green');
ax[3].plot(eth['Low'], color='orange');
ax[4].plot(eth['Mean'], color='cyan');

ax[5].plot(eth['Volatility'], color='black');
ax[6].plot(eth['Log_Returns'], color='purple');

### LTC

In [ ]:
ltc = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/LTC.csv')
ltc.head()

,Crypto,Date,Open,High,Low,Close
0,LTC,01/01/2018,230.32,237.77,217.87,225.63
1,LTC,02/01/2018,162.33,164.87,127.00,140.84
2,LTC,03/01/2018,201.73,215.50,200.30,209.05
3,LTC,04/01/2018,115.85,117.59,109.33,115.00
4,LTC,05/01/2018,149.91,150.15,143.22,145.72


In [ ]:
# --------------------------------
# Step 1 — Sort by ascending dates
# --------------------------------

ltc['Date'] = pd.to_datetime(ltc['Date'], format='mixed', dayfirst=False)
ltc = ltc.sort_values(by='Date', ascending=True)
ltc['Date'] = ltc['Date'].dt.strftime('%d/%m/%Y')

In [ ]:
ltc.head()
### 1,975 instead of 1,977
ltc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1975 entries, 0 to 1604
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Crypto  1975 non-null   object 
 1   Date    1975 non-null   object 
 2   Open    1975 non-null   float64
 3   High    1975 non-null   float64
 4   Low     1975 non-null   float64
 5   Close   1975 non-null   float64
dtypes: float64(4), object(2)
memory usage: 108.0+ KB


In [ ]:
# -------------------------------------
# Step 2 — Determine missing date/value
# -------------------------------------

ltc['Date'] = pd.to_datetime(ltc['Date'], dayfirst=True)
full_range = pd.date_range(start='2018-01-01', end='2023-05-31')
diff = full_range.difference(ltc['Date'])
print("Missing dates in the dataset:", diff)
# ltc: 12/01/2018, 02/12/2020 missing values

Missing dates in the dataset: DatetimeIndex(['2018-01-12', '2020-12-02'], dtype='datetime64[ns]', freq=None)


In [ ]:
# --------------------------
# Step 3 — Add missing dates
# --------------------------

ltc.set_index('Date', inplace=True)
ltc = ltc.reindex(full_range)

In [ ]:
# ----------------------------------------------------
# Step 4 — Linearly interpolate missing values (price)
# ----------------------------------------------------

ltc['Crypto']= 'LTC'
ltc.isnull().sum()
ltc.interpolate(method='linear', inplace=True)
ltc.reset_index(inplace=True)
ltc.rename(columns={'index': 'Date'}, inplace=True)

/tmp/ipython-input-3633247927.py:7: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  ltc.interpolate(method='linear', inplace=True)


In [ ]:
# -----------------------------------------
# Step 5 — Export new DF (without scaling)
# -----------------------------------------

ltc.to_csv('LTC_noscale.csv', index=False)

from google.colab import files
files.download('LTC_noscale.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# -----------------------------------------------
# Step 6  — Plot Dataset Features over the years
# -----------------------------------------------
# Plot Target variable i.e. LTC close price
plt.figure(figsize=(10,5))
plt.plot(ltc['Close'], color='blue');
figure, ax = plt.subplots(nrows=7, ncols=1, figsize=(12,10))
ax[0].set_ylabel('Close Price')
ax[1].set_ylabel('Open Price')
ax[2].set_ylabel('High Price')
ax[3].set_ylabel('Low Price')
ax[4].set_ylabel('Mean Price')
ax[5].set_ylabel('Volatility')
ax[6].set_ylabel('Log Returns')

ax[0].plot(ltc['Close'], color='blue');
ax[1].plot(ltc['Open'], color='red', alpha=0.75);
ax[2].plot(ltc['High'], color='green');
ax[3].plot(ltc['Low'], color='orange');
ax[4].plot(ltc['Mean'], color='cyan');

ax[5].plot(ltc['Volatility'], color='black');
ax[6].plot(ltc['Log_Returns'], color='purple');

### XRP

In [ ]:
xrp = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/XRP.csv')
xrp.head()

,Crypto,Date,Open,High,Low,Close
0,XRP,12/31/22,0.34,0.34,0.34,0.34
1,XRP,12/30/22,0.34,0.35,0.33,0.34
2,XRP,12/29/22,0.36,0.36,0.34,0.34
3,XRP,12/28/22,0.37,0.37,0.35,0.36
4,XRP,12/27/22,0.37,0.37,0.36,0.37


In [ ]:
# --------------------------------
# Step 1 — Sort by ascending dates
# --------------------------------

xrp['Date'] = pd.to_datetime(xrp['Date'], dayfirst=False)
xrp = xrp.sort_values(by='Date', ascending=True)
xrp['Date'] = xrp['Date'].dt.strftime('%d/%m/%Y')

/tmp/ipython-input-2211538312.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  xrp['Date'] = pd.to_datetime(xrp['Date'], dayfirst=False)


In [ ]:
xrp.tail()
## 1,975 datapoints instead of 1,977
xrp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1975 entries, 1609 to 365
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Crypto  1975 non-null   object 
 1   Date    1975 non-null   object 
 2   Open    1975 non-null   float64
 3   High    1975 non-null   float64
 4   Low     1975 non-null   float64
 5   Close   1975 non-null   float64
dtypes: float64(4), object(2)
memory usage: 108.0+ KB


In [ ]:
# -------------------------------------
# Step 2 — Determine missing date/value
# -------------------------------------

xrp['Date'] = pd.to_datetime(xrp['Date'], dayfirst=True)
full_range = pd.date_range(start='2018-01-01', end='2023-05-31')
diff = full_range.difference(xrp['Date'])

print("Missing dates in the dataset:", diff)
# xrp: 12/01/2018, 02/12/2020 missing values

Missing dates in the dataset: DatetimeIndex(['2018-01-12', '2020-12-02'], dtype='datetime64[ns]', freq=None)


In [ ]:
# --------------------------
# Step 3 — Add missing dates
# --------------------------

xrp.set_index('Date', inplace=True)
xrp = xrp.reindex(full_range)

In [ ]:
# ----------------------------------------------------
# Step 4 — Linearly interpolate missing values (price)
# ----------------------------------------------------

xrp['Crypto']= 'LTC'
xrp.isnull().sum()
xrp.interpolate(method='linear', inplace=True)
xrp.reset_index(inplace=True)
xrp.rename(columns={'index': 'Date'}, inplace=True)

/tmp/ipython-input-196699441.py:7: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  xrp.interpolate(method='linear', inplace=True)


In [ ]:
# -----------------------------------------
# Step 5 — Export new DF (without scaling)
# -----------------------------------------

xrp.to_csv('XRP_noscale.csv', index=False)

from google.colab import files
files.download('XRP_noscale.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# -----------------------------------------------
# Step 6  — Plot Dataset Features over the years
# -----------------------------------------------
# Plot Target variable i.e. XRP close price
plt.figure(figsize=(10,5))
plt.plot(xrp['Close'], color='blue');
figure, ax = plt.subplots(nrows=7, ncols=1, figsize=(12,10))
ax[0].set_ylabel('Close Price')
ax[1].set_ylabel('Open Price')
ax[2].set_ylabel('High Price')
ax[3].set_ylabel('Low Price')
ax[4].set_ylabel('Mean Price')
ax[5].set_ylabel('Volatility')
ax[6].set_ylabel('Log Returns')

ax[0].plot(xrp['Close'], color='blue');
ax[1].plot(xrp['Open'], color='red', alpha=0.75);
ax[2].plot(xrp['High'], color='green');
ax[3].plot(xrp['Low'], color='orange');
ax[4].plot(xrp['Mean'], color='cyan');

ax[5].plot(xrp['Volatility'], color='black');
ax[6].plot(xrp['Log_Returns'], color='purple');